In [1]:
from seamless.highlevel import Context, Transformer, Cell

ctx = Context()
ctx.auto_translate = True

In [2]:
ctx.pdb1 = open("1AKE-flanks.pdb").read()
ctx.pdb2 = open("1AKE-B-hit.pdb").read()
ctx.load_pdb1 = Transformer()
ctx.load_pdb1.pdb = ctx.pdb1
ctx.load_pdb_code >> ctx.load_pdb1.code
ctx.load_pdb_code.mount("load_pdb.py")
ctx.coor1 = ctx.load_pdb1
ctx.load_pdb2 = Transformer()
ctx.load_pdb2.pdb = ctx.pdb2
ctx.load_pdb2.code = ctx.load_pdb_code
ctx.coor2 = ctx.load_pdb2

Qt could not be started. Qt widgets will not work


In [3]:
def bcscore(coor1, coor2, flanksize, gapsize):
    print("BCSCORE", flanksize, gapsize)
    return 0.5

ctx.bcscore = bcscore
ctx.bcscore.coor1 = ctx.coor1
ctx.bcscore.coor2 = ctx.coor2
ctx.bcscore.flanksize = 4
ctx.bcscore.gapsize = 7

ctx.bc_result = ctx.bcscore
await ctx.computation()
print(ctx.bc_result.value)

<Silk: 0.5 >


In [4]:
ctx.bcscore.language = "c"

ctx.bcscore_schema = Cell()
ctx.bcscore_schema.celltype = "plain"
ctx.bcscore_schema.mount("bcscore-schema.json", "r", authority="file")
await ctx.translation()
ctx.bcscore.result.example.set(1.0)
ctx.bcscore.result.schema["form"] = {"bytesize": 8}
await ctx.translation()
ctx.link(ctx.bcscore_schema, ctx.bcscore.inp.schema)
ctx.header = ctx.bcscore.header
ctx.header.mimetype = "h"
await ctx.translation()

In [5]:
ctx.bcscore.header.mount("bcscore.h", "w")
ctx.header.output_widget()

Output()

In [6]:
ctx.bcscore.code.mount("bcscore.c", authority="file")
ctx.bcscore.result.example.set(1.0)
ctx.bcscore.result.schema["form"] = {"bytesize": 4}
ctx.bcscore.main_module.bclib.language = "c"
ctx.bclib_code >> ctx.bcscore.main_module.bclib.code 
ctx.bclib_code.mount("BClib.c")
await ctx.translation()


In [7]:
await ctx.computation()
print(ctx.bcscore.inp.exception)
print(ctx.bcscore.exception)

None
None


In [8]:
import numpy as np

def validate_shape(self):
    assert self.coor1.shape[0] == 2 * self.flanksize, self.coor1.shape
    assert self.coor2.shape[0] == 2 * self.flanksize + self.gapsize, self.coor2.shape

example = ctx.bcscore.example
example.coor1 = np.zeros((10, 3))
example.coor2 = np.zeros((14, 3))
example.flanksize = 5
example.gapsize = 4
example.add_validator(validate_shape, name="validate_shape")

In [9]:
await ctx.computation()
print(ctx.bcscore.schema)
print(ctx.bcscore.result.schema)

{'properties': {'coor1': {'form': {'contiguous': True, 'ndim': 2, 'shape': [-1, 3]}, 'items': {'form': {'bytesize': 8, 'type': 'number'}}, 'storage': 'binary', 'type': 'array'}, 'coor2': {'form': {'contiguous': True, 'ndim': 2, 'shape': [-1, 3]}, 'items': {'form': {'bytesize': 8, 'type': 'number'}}, 'storage': 'binary', 'type': 'array'}, 'flanksize': {'type': 'integer'}, 'gapsize': {'type': 'integer'}}, 'type': 'object', 'validators': [{'code': 'def validate_shape(self):\n    assert self.coor1.shape[0] == 2 * self.flanksize, self.coor1.shape\n    assert self.coor2.shape[0] == 2 * self.flanksize + self.gapsize, self.coor2.shape\n', 'language': 'python', 'name': 'validate_shape'}]}
{'form': {'bytesize': 4}, 'type': 'number'}


In [10]:
await ctx.computation()
print(ctx.bc_result.value)

<Silk: 0.9981744289398193 >


In [11]:
ctx.save_graph("bcscore.json")
ctx.save_zip("bcscore.zip")

NameError: name 'json' is not defined